In [1]:
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import h5py
import numba as nb
from numba import jit, njit
import tracerFuncs as tF
import insituFuncs as iF
import funcs

plt.style.use('default')
plt.rcParams["figure.figsize"][0] = 16
plt.rcParams["figure.figsize"][1] = 9
plt.rcParams['errorbar.capsize']=2

In [2]:
basePath='/virgotng/universe/IllustrisTNG/TNG50-4/output'
h_const=il.groupcat.loadHeader(basePath,99)['HubbleParam']
boxSize = il.groupcat.loadHeader(basePath,99)['BoxSize']
print(h_const)

0.6774


do traceback with different redshifts:

In [3]:
%%time
tF.TraceBackAllInsituStars(basePath,99,13)

gas:
False
# matches:  273367
# non-matches:  2173
stars:
False
# matches:  2174
# non-matches:  273366


ValueError: shape mismatch: value array of shape (2174,) could not be broadcast to indexing result of shape (2173,)

# test:

In [48]:
all_target_gas_ids = il.snapshot.loadSubset(basePath,99,0,['ParticleIDs'])
all_target_star_ids = il.snapshot.loadSubset(basePath,99,4,['ParticleIDs'])

In [52]:
def match(ar1, ar2, is_sorted = False):
    """ Returns index arrays i1,i2 of the matching elements between ar1 and ar2. While the elements of ar1 
        must be unique, the elements of ar2 need not be. For every matched element of ar2, the return i1 
        gives the index in ar1 where it can be found. For every matched element of ar1, the return i2 gives 
        the index in ar2 where it can be found. Therefore, ar1[i1] = ar2[i2]. The order of ar2[i2] preserves 
        the order of ar2. Therefore, if all elements of ar2 are in ar1 (e.g. ar1=all TracerIDs in snap, 
        ar2=set of TracerIDs to locate) then ar2[i2] = ar2. The approach is one sort of ar1 followed by 
        bisection search for each element of ar2, therefore O(N_ar1*log(N_ar1) + N_ar2*log(N_ar1)) ~= 
        O(N_ar1*log(N_ar1)) complexity so long as N_ar2 << N_ar1. """
    
    if not is_sorted:
        # need a sorted copy of ar1 to run bisection against
        index = np.argsort(ar1)
        ar1_sorted = ar1[index]
    else:
        ar1_sorted = ar1
        index = np.arange(ar1.shape[0])
    # NlogN search of ar1_sorted for each element in ar2
    ar1_sorted_index = np.searchsorted(ar1_sorted, ar2)

    # undo sort
    ar1_inds = np.take(index, ar1_sorted_index, mode="clip")

    # filter out non-matches
    mask = (ar1[ar1_inds] == ar2)
    ar2_inds = np.where(mask)[0]
    
    #fill non-matches with -1
    ar1_inds[np.where(np.logical_not(mask))[0]] = -1
    #ar1_inds = ar1_inds[ar2_inds]

    return ar1_inds, ar2_inds

In [66]:
def parentIndicesOfAll(parent_ids, all_gas_ids, all_star_ids): 
    #which parent corresponds to which gas/star particles?
    #for this: save index into gas('0') / star('1') subset array
    target_parent_indices = np.zeros((len(parent_ids),2))
    gas_inds, _ = match(all_gas_ids,parent_ids) #only inds1 relevant
    star_inds, _ = match(all_star_ids,parent_ids) #only inds1 relevant
    
    #gas_inds now contains either the indices of gas parents into the gas subset or -1; same for star_inds
    target_parent_indices[:,0] = gas_inds
    target_parent_indices[:,1] = 0
    target_parent_indices[np.where(gas_inds == -1)[0],1] = 1 #no gas index found => parent is a star
    target_parent_indices[np.where(gas_inds == -1)[0],0] = star_inds[np.where(star_inds!=-1)[0]]
    del gas_inds, star_inds
    return target_parent_indices

In [72]:
%%time
a = parentIndicesOfAll(all_target_gas_ids,all_target_gas_ids,all_target_star_ids)

CPU times: user 1min 29s, sys: 1.74 s, total: 1min 31s
Wall time: 1min 31s


In [74]:
test = np.where(a[:,1]==1)[0]
print(test)

[]


In [4]:
ar2 = np.array([1,1,4,3,0,4]) #parent_ids
ar1 = np.array([4,1,0]) #gas_ids
inds1, inds2 = tF.match(ar1,ar2)
print(np.ravel(ar1)[np.array([0,2,1])]) #alternative to np.take

False
# matches:  5
# non-matches:  1
[4 0 1]


In [5]:
print(inds1)
print(inds2)

[ 1  1  0 -1  2  0]
[0 1 2 4 5]


In [7]:
#define function that saves results from TraceAllStars now for every single snapshot
def TraceBackAllInsituStars_allSnaps(basePath,start_snap):
    #load all star ids from a specific galaxy
    star_ids = il.snapshot.loadSubset(basePath,start_snap,'stars',fields=['ParticleIDs'])

    #determine all stars from that galaxy that were formed insitu
    insitu = iF.is_insitu(basePath,np.arange(star_ids.size),start_snap)
    insitu_star_indices = np.nonzero(insitu)[0]
    
    insituStarsInSubOffset = tF.insituStarsInSubOffset(basePath, start_snap)
    
    sim = basePath[32:39]    
    result = h5py.File('files/'+sim+'/all_parent_indices.hdf5','w')    
    
    #run function for every snapshot
    for target_snap in np.arange(99,1,-1):
        parent_indices, tracersInSubOffset = tF.TraceAllStars(basePath,star_ids[insitu_star_indices],\
                                                       start_snap,target_snap,insituStarsInSubOffset)  
        #save results in hdf5 file

        grp = result.create_group(f'snap_0{target_snap}')
        dset = grp.create_dataset("parent_indices", parent_indices.shape, dtype=float)
        dset[:] = parent_indices
        dset2 = grp.create_dataset('tracers_in_parents_offset',tracersInSubOffset.shape, dtype=float)
        dset2[:] = tracersInSubOffset
        print(target_snap, 'done',flush=True)
    result.close()
    return

In [8]:
basePath='/virgotng/universe/IllustrisTNG/TNG50-4/output'
TraceBackAllInsituStars_allSnaps(basePath,start_snap = 99)

99 done
98 done
97 done
96 done
95 done
94 done
93 done
92 done
91 done
90 done
89 done
88 done
87 done
86 done
85 done
84 done
83 done
82 done
81 done
80 done
79 done
78 done
77 done
76 done
75 done
74 done
73 done
72 done
71 done
70 done
69 done
68 done
67 done
66 done
65 done
64 done
63 done
62 done
61 done
60 done
59 done
58 done
57 done
56 done
55 done
54 done
53 done
52 done
51 done
50 done
49 done
48 done
47 done
46 done
45 done
44 done
43 done
42 done
41 done
40 done
39 done
38 done
37 done
36 done
35 done
34 done
33 done
32 done
31 done
30 done
29 done
28 done
27 done
26 done
25 done
24 done
23 done
22 done
21 done
20 done
19 done
18 done
17 done
16 done
15 done
14 done


ValueError: shape mismatch: value array of shape (2174,) could not be broadcast to indexing result of shape (2173,)

In [9]:
f = h5py.File('files/TNG50-2/all_parent_indices.hdf5','r')
print(f.keys())
f.close()

<KeysViewHDF5 ['snap_010', 'snap_011', 'snap_012', 'snap_013', 'snap_014', 'snap_015', 'snap_016', 'snap_017', 'snap_018', 'snap_019', 'snap_02', 'snap_020', 'snap_021', 'snap_022', 'snap_023', 'snap_024', 'snap_025', 'snap_026', 'snap_027', 'snap_028', 'snap_029', 'snap_03', 'snap_030', 'snap_031', 'snap_032', 'snap_033', 'snap_034', 'snap_035', 'snap_037', 'snap_038', 'snap_039', 'snap_04', 'snap_040', 'snap_041', 'snap_042', 'snap_043', 'snap_044', 'snap_045', 'snap_046', 'snap_047', 'snap_048', 'snap_049', 'snap_05', 'snap_050', 'snap_051', 'snap_052', 'snap_053', 'snap_054', 'snap_055', 'snap_056', 'snap_057', 'snap_058', 'snap_059', 'snap_06', 'snap_060', 'snap_061', 'snap_062', 'snap_063', 'snap_064', 'snap_065', 'snap_066', 'snap_067', 'snap_068', 'snap_069', 'snap_07', 'snap_070', 'snap_071', 'snap_072', 'snap_073', 'snap_074', 'snap_075', 'snap_076', 'snap_077', 'snap_078', 'snap_079', 'snap_08', 'snap_080', 'snap_081', 'snap_082', 'snap_083', 'snap_084', 'snap_085', 'snap_08